In [0]:
CREATE OR REPLACE TABLE some_catalog.gold.cross_city_comparison AS
SELECT
  (SELECT ROUND(AVG(price), 2) FROM some_catalog.silver.listing WHERE is_berlin = true)  AS berlin_avg_price,
  (SELECT ROUND(AVG(price), 2) FROM some_catalog.silver.listing WHERE is_berlin = false) AS stockholm_avg_price,
  (SELECT ROUND(AVG(number_of_reviews), 2) FROM some_catalog.silver.listing WHERE is_berlin = true)  AS berlin_avg_reviews,
  (SELECT ROUND(AVG(number_of_reviews), 2) FROM some_catalog.silver.listing WHERE is_berlin = false) AS stockholm_avg_reviews,
  (SELECT COUNT(DISTINCT listing_id) FROM some_catalog.silver.listing WHERE is_berlin = true)  AS berlin_listings,
  (SELECT COUNT(DISTINCT listing_id) FROM some_catalog.silver.listing WHERE is_berlin = false) AS stockholm_listings;




In [0]:
%python
# %python  ← (залиш, якщо виконуєш у SQL-ноуті з magic; у звичайному Python-ноуті не потрібно)

# ================================================================
# ONE-CELL ANALYSIS (працює з твоїми таблицями в some_catalog.airbnb_gold)
# ================================================================
# Віджети (поміняй за потреби)
dbutils.widgets.text("catalog", "some_catalog")
dbutils.widgets.text("schema_city_tables", "airbnb_gold")   # тут лежать berlin_* / stock_*
dbutils.widgets.text("cities_csv", "Stock,Berlin")          # два міста (суфікси: Stock -> stock_*, Berlin -> berlin_*)

CAT   = dbutils.widgets.get("catalog")
SCH   = dbutils.widgets.get("schema_city_tables")
CITIES= [c.strip() for c in dbutils.widgets.get("cities_csv").split(",") if c.strip()]

from pyspark.sql import functions as F, types as T

# ---- утиліти ---------------------------------------------------
def tbl(cat, sch, name): return f"{cat}.{sch}.{name}"
def cfix(city): return city.replace(" ", "_")

# мапа "City" -> префікс у назвах таблиць
CITY_PREFIX = {c: c.lower() for c in CITIES}  # "Stock"->"stock", "Berlin"->"berlin"

def exists_table(fullname: str) -> bool:
    try:
        spark.table(fullname).limit(1).count()
        return True
    except Exception:
        return False

def parse_money(col):
    return F.regexp_replace(F.col(col).cast("string"), r"[^0-9.]", "").cast("double")

def missing_report(df):
    if df is None: 
        return None
    total = df.count()
    if total == 0:
        return spark.createDataFrame([], "column string, total_rows long, missing_count long, missing_percent double")
    sums = [F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]
    miss = df.agg(*sums)
    stack = "stack({n}, {pairs}) as (column, missing_count)".format(
        n=len(df.columns),
        pairs=", ".join([f"'{c}', `{c}`" for c in df.columns])
    )
    return (miss.select(F.expr(stack))
                .withColumn("total_rows", F.lit(total))
                .withColumn("missing_percent", F.round(100*F.col("missing_count")/F.col("total_rows"),2)))

# ---- зчитування міських таблиць --------------------------------
def load_listing(city):
    prefix = CITY_PREFIX[city]
    name = tbl(CAT, SCH, f"{prefix}_listing")
    if not exists_table(name): return None
    df = spark.table(name)
    if "price" in df.columns and "price_num" not in df.columns:
        df = df.withColumn("price_num", parse_money("price"))
    if "availability_365" in df.columns and "occ_rate_365" not in df.columns:
        df = df.withColumn("occ_rate_365", 1.0 - (F.col("availability_365").cast("double")/365.0))
    return df.withColumn("city", F.lit(city))

def load_detailed_listing(city):
    prefix = CITY_PREFIX[city]
    name = tbl(CAT, SCH, f"{prefix}_detailed_listing")
    if not exists_table(name): return None
    df = spark.table(name)
    if "price" in df.columns and "price_num" not in df.columns:
        df = df.withColumn("price_num", parse_money("price"))
    # дати скрейпу до дат
    for c in ["last_scraped", "calendar_last_scraped", "first_review", "last_review"]:
        if c in df.columns:
            df = df.withColumn(c, F.to_date(F.col(c)))
    return df.withColumn("city", F.lit(city))

def load_calendar(city):
    prefix = CITY_PREFIX[city]
    name = tbl(CAT, SCH, f"{prefix}_calendar")
    if not exists_table(name): return None
    df = spark.table(name)
    # date → date
    if "date" in df.columns:
        df = df.withColumn("date", F.to_date(F.col("date")))
    # available 't'/'f' → boolean
    if "available" in df.columns:
        df = df.withColumn(
            "available",
            F.when(F.lower(F.col("available").cast("string"))=="t", True)
             .when(F.lower(F.col("available").cast("string"))=="f", False)
             .otherwise(None)
        )
    return df.withColumn("city", F.lit(city))

def load_reviews(city):
    # беремо detailed_reviews якщо є, інакше reviews
    prefix = CITY_PREFIX[city]
    cand = [f"{prefix}_detailed_reviews", f"{prefix}_reviews"]
    for n in cand:
        full = tbl(CAT, SCH, n)
        if exists_table(full):
            return spark.table(full).withColumn("city", F.lit(city))
    return None

# ---- побудова агрегатів "на льоту" ------------------------------
city_metrics_rows = []
occ_union = None
price_hist_union = None
nb_union = None
datasets_desc = []

for CITY in CITIES:
    lst = load_listing(CITY)
    dlt = load_detailed_listing(CITY)
    cal = load_calendar(CITY)
    rev = load_reviews(CITY)

    # 1) City metrics (listing_count, host_count, median/avg price, avg_occ_365, avg_review_score)
    if lst is not None:
        aggs = [
            F.countDistinct("id").alias("listing_count"),
            F.countDistinct("host_id").alias("host_count") if "host_id" in lst.columns else F.lit(None).alias("host_count"),
            F.expr("percentile_approx(price_num,0.5)").alias("median_price") if "price_num" in lst.columns else F.lit(None).alias("median_price"),
            F.avg("price_num").alias("avg_price") if "price_num" in lst.columns else F.lit(None).alias("avg_price"),
            F.avg("occ_rate_365").alias("avg_occ_365") if "occ_rate_365" in lst.columns else F.lit(None).alias("avg_occ_365"),
        ]
        cm = lst.agg(*aggs).withColumn("city", F.lit(CITY))
        # середній рейтинг із detailed_listing
        if dlt is not None and "review_scores_rating" in dlt.columns:
            cm = cm.crossJoin(dlt.agg(F.avg("review_scores_rating").alias("avg_review_score")))
        else:
            cm = cm.withColumn("avg_review_score", F.lit(None))
        city_metrics_rows.append(cm.select("city","listing_count","host_count","median_price","avg_price","avg_occ_365","avg_review_score"))
        datasets_desc.append(("listing", CITY, lst.count(), None, None))

    if dlt is not None:
        datasets_desc.append(("detailed_listing", CITY, dlt.count(),
                              dlt.agg(F.min("last_scraped")).first()[0] if "last_scraped" in dlt.columns else None,
                              dlt.agg(F.max("last_scraped")).first()[0] if "last_scraped" in dlt.columns else None))

    # 2) Occupancy by month (із calendar)
    if cal is not None and "date" in cal.columns and "available" in cal.columns:
        occ = (cal.groupBy("city", F.date_trunc("month", F.col("date")).alias("month"))
                   .agg(F.avg(F.when(F.col("available")==False, 1).otherwise(0)).alias("occ_rate_month")))
        occ_union = occ if occ_union is None else occ_union.unionByName(occ, True)
        datasets_desc.append(("calendar", CITY, cal.count(),
                              cal.agg(F.min("date")).first()[0], cal.agg(F.max("date")).first()[0]))

    # 3) Price histogram (із detailed_listing — снапшоти)
    if dlt is not None and "price_num" in dlt.columns:
        ph = (dlt.where((F.col("price_num")>=5) & (F.col("price_num")<=2000))
                 .withColumn("price_bucket", F.round("price_num"))
                 .groupBy("city","price_bucket").count())
        price_hist_union = ph if price_hist_union is None else price_hist_union.unionByName(ph, True)

    # 4) Neighbourhood metrics (із listing)
    if lst is not None and "neighbourhood_cleansed" in lst.columns:
        nb = (lst.where(F.col("neighbourhood_cleansed").isNotNull())
                .groupBy("city", F.col("neighbourhood_cleansed").alias("neighbourhood"))
                .agg(F.count("*").alias("listing_cnt"),
                     F.avg("price_num").alias("avg_price"),
                     F.avg("occ_rate_365").alias("avg_occ_365"),
                     F.avg("latitude").alias("lat"),
                     F.avg("longitude").alias("lon")))
        nb_union = nb if nb_union is None else nb_union.unionByName(nb, True)

# ---- виводи -----------------------------------------------------
from functools import reduce

# CLEAN SUMMARY
if city_metrics_rows:
    cm_all = reduce(lambda a,b: a.unionByName(b, True), city_metrics_rows)
    # додаємо p10/median/p90 зайнятості за місяцями
    if occ_union is not None:
        occ_stats = (occ_union.groupBy("city")
                     .agg(F.avg("occ_rate_month").alias("occ_month_avg"),
                          F.expr("percentile_approx(occ_rate_month,0.5)").alias("occ_month_median"),
                          F.expr("percentile_approx(occ_rate_month,0.1)").alias("occ_p10"),
                          F.expr("percentile_approx(occ_rate_month,0.9)").alias("occ_p90")))
        clean_summary = cm_all.join(occ_stats, on="city", how="left")
    else:
        clean_summary = cm_all
    print("### CLEAN SUMMARY (per city)")
    display(clean_summary.orderBy("city"))
else:
    print("⚠️ Не знайдено жодної таблиці listing_<city> у", tbl(CAT, SCH, "*"))

# DESCRIBE DATASETS
if datasets_desc:
    ds = spark.createDataFrame(datasets_desc, ["dataset","city","rows","min_date","max_date"])
    print("### DATASETS — наявність і обсяги")
    display(ds.orderBy("dataset","city"))
else:
    print("ℹ️ Нема що описувати — перевір наявність berlin_* / stock_* в", tbl(CAT, SCH, ""))

# MISSING RATES (компактні таблиці)
miss_parts = []
for (name, df) in [("neighbourhood_metrics", nb_union),
                   ("price_hist", price_hist_union),
                   ("occ_by_month", occ_union)]:
    if df is not None:
        m = missing_report(df).withColumn("table", F.lit(name))
        miss_parts.append(m)

if miss_parts:
    miss_union = reduce(lambda a,b: a.unionByName(b, True), miss_parts)
    print("### Missing value rates (компактні агрегати)")
    display(miss_union.orderBy("table","missing_percent", ascending=[True, False]))
else:
    print("ℹ️ Нема що рахувати для Missing (ще не зібрались компактні агрегати).")

# DISTRIBUTIONS
if price_hist_union is not None:
    print("### Price distribution (histogram)")
    display(price_hist_union.orderBy("city","price_bucket"))
else:
    print("ℹ️ Нема даних для гістограми цін (detailed_listing без price або відфільтровано все).")

if occ_union is not None:
    print("### Occupancy by month")
    display(occ_union.orderBy("city","month"))
else:
    print("ℹ️ Нема даних для помісячної зайнятості (calendar відсутній або без available/date).")

# REQUIRED CLEANING & TRANSFORMATIONS (авто-рекомендації)
recs = []

# 1) Якість price histogram
if price_hist_union is None or price_hist_union.count() < 10:
    for c in CITIES:
        recs.append((c, "price_hist", "Мало/нема бінів: перевір detailed_listing.price → price_num та діапазон [5..2000]."))

# 2) Пропуски в neighbourhood_metrics
if nb_union is None or nb_union.count()==0:
    for c in CITIES:
        recs.append((c, "neighbourhood_metrics", "Порожньо/нема районів: перевір listing.neighbourhood_cleansed/geo."))
else:
    mm = missing_report(nb_union.select("lat","lon","avg_price","avg_occ_365"))
    d = {r["column"]: r["missing_percent"] for r in mm.collect()}
    for k in ["lat","lon","avg_price","avg_occ_365"]:
        v = d.get(k, 0)
        if v and v > 20:
            recs.append((", both cities", "neighbourhood_metrics", f"{k} має {v:.1f}% NULL — фільтруй/імпутуй перед heatmap."))

# 3) Покриття по місяцях
if occ_union is None or occ_union.count()==0:
    for c in CITIES:
        recs.append((c, "occ_by_month", "Нема помісячної зайнятості: перевір calendar.date та available ('t'/'f')."))
else:
    months_by_city = (occ_union.groupBy("city").agg(F.countDistinct("month").alias("n_months"))).collect()
    for r in months_by_city:
        if r["n_months"] < 6:
            recs.append((r["city"], "occ_by_month", f"Коротка історія ({r['n_months']} міс.) — сезонність слабка; додай дані/агрегуй поквартально."))

print("### REQUIRED CLEANING & TRANSFORMATIONS")
if recs:
    display(spark.createDataFrame(recs, ["city","dataset","recommendation"]))
else:
    print("✅ Явних проблем не виявлено.")


In [0]:
%python
# %python  # лишай, якщо ти в SQL-ноуті

# ---------- ПАРАМЕТРИ ----------
dbutils.widgets.text("catalog", "some_catalog")
dbutils.widgets.text("schema_city_tables", "airbnb_gold")
dbutils.widgets.text("cities_csv", "Stock,Berlin")

CAT = dbutils.widgets.get("catalog")
SCH = dbutils.widgets.get("schema_city_tables")
CITIES = [c.strip() for c in dbutils.widgets.get("cities_csv").split(",") if c.strip()]

from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd

def T(name): return f"{CAT}.{SCH}.{name}"
CITY_PREFIX = {c: c.lower() for c in CITIES}

def exists_table(full):
    try:
        spark.table(full).limit(1).count()
        return True
    except:
        return False

def parse_money(col):
    return F.regexp_replace(F.col(col).cast("string"), r"[^0-9.]", "").cast("double")

# ---------- ЗАВАНТАЖЕННЯ ----------
def load_listing(city):
    full = T(f"{CITY_PREFIX[city]}_listing")
    if not exists_table(full): return None
    df = spark.table(full)
    if "price" in df.columns and "price_num" not in df.columns:
        df = df.withColumn("price_num", parse_money("price"))
    if "availability_365" in df.columns and "occ_rate_365" not in df.columns:
        df = df.withColumn("occ_rate_365", 1.0 - (F.col("availability_365").cast("double")/365.0))
    return df.withColumn("city", F.lit(city))

def load_detailed(city):
    full = T(f"{CITY_PREFIX[city]}_detailed_listing")
    if not exists_table(full): return None
    df = spark.table(full)
    if "price" in df.columns and "price_num" not in df.columns:
        df = df.withColumn("price_num", parse_money("price"))
    for c in ["last_scraped", "calendar_last_scraped", "first_review", "last_review"]:
        if c in df.columns:
            df = df.withColumn(c, F.to_date(F.col(c)))
    return df.withColumn("city", F.lit(city))

def load_calendar(city):
    full = T(f"{CITY_PREFIX[city]}_calendar")
    if not exists_table(full): return None
    df = spark.table(full)
    if "date" in df.columns:
        df = df.withColumn("date", F.to_date("date"))
    if "available" in df.columns:
        df = df.withColumn(
            "available",
            F.when(F.lower(F.col("available").cast("string"))=="t", True)
             .when(F.lower(F.col("available").cast("string"))=="f", False)
             .otherwise(None)
        )
    return df.withColumn("city", F.lit(city))

# ---------- АГРЕГАТИ ДЛЯ ГРАФІКІВ ----------
price_hist = None
occ_month = None
room_share = None
rating_hist = None
summary_rows = []

for city in CITIES:
    lst = load_listing(city)
    dlt = load_detailed(city)
    cal = load_calendar(city)

    # Clean summary: listing/host, avg/median price, avg_occ, avg rating
    if lst is not None:
        agg = lst.agg(
            F.countDistinct("id").alias("listing_count"),
            (F.countDistinct("host_id") if "host_id" in lst.columns else F.lit(None)).alias("host_count"),
            (F.expr("percentile_approx(price_num,0.5)") if "price_num" in lst.columns else F.lit(None)).alias("median_price"),
            (F.avg("price_num") if "price_num" in lst.columns else F.lit(None)).alias("avg_price"),
            (F.avg("occ_rate_365") if "occ_rate_365" in lst.columns else F.lit(None)).alias("avg_occ_365"),
        ).first()
        avg_rating = None
        if dlt is not None and "review_scores_rating" in dlt.columns:
            avg_rating = dlt.agg(F.avg("review_scores_rating")).first()[0]
        summary_rows.append([city, agg["listing_count"], agg["host_count"], agg["median_price"], agg["avg_price"], agg["avg_occ_365"], avg_rating])

    # Price histogram (з detailed)
    if dlt is not None and "price_num" in dlt.columns:
        ph = (dlt.where((F.col("price_num")>=5) & (F.col("price_num")<=2000))
                 .withColumn("price_bucket", F.round("price_num"))
                 .groupBy(F.lit(city).alias("city"), "price_bucket").count())
        price_hist = ph if price_hist is None else price_hist.unionByName(ph, True)

    # Occupancy by month (з calendar)
    if cal is not None and "date" in cal.columns and "available" in cal.columns:
        om = (cal.groupBy(F.lit(city).alias("city"), F.date_trunc("month", "date").alias("month"))
                 .agg(F.avg(F.when(F.col("available")==False, 1).otherwise(0)).alias("occ_rate_month")))
        occ_month = om if occ_month is None else occ_month.unionByName(om, True)

    # Room type share (з listing)
    if lst is not None and "room_type" in lst.columns:
        rs = (lst.groupBy(F.lit(city).alias("city"), "room_type").count())
        total = rs.agg(F.sum("count")).first()[0]
        rs = rs.withColumn("share", F.col("count")/F.lit(total))
        room_share = rs if room_share is None else room_share.unionByName(rs, True)

    # Ratings histogram (з detailed)
    if dlt is not None and "review_scores_rating" in dlt.columns:
        rh = (dlt.where(F.col("review_scores_rating").isNotNull())
                .withColumn("rating_bin", F.floor(F.col("review_scores_rating")/5.0)*5)
                .groupBy(F.lit(city).alias("city"), "rating_bin").count())
        rating_hist = rh if rating_hist is None else rating_hist.unionByName(rh, True)

# ---------- CLEAN SUMMARY (таблиця) ----------
if summary_rows:
    import pandas as pd
    df_summary = pd.DataFrame(summary_rows, columns=["city","listing_count","host_count","median_price","avg_price","avg_occ_365","avg_review_score"])
    print("### CLEAN SUMMARY (пер місто)")
    display(spark.createDataFrame(df_summary))
else:
    print("⚠️ Не вдалося зібрати summary — перевір наявність *_listing таблиць.")

# ---------- PLOTS ----------
plt.close('all')

# 1) Price histogram (лінії по містах)
if price_hist is not None and price_hist.count() > 0:
    pdf = price_hist.toPandas().sort_values(["city","price_bucket"])
    pivot = pdf.pivot(index="price_bucket", columns="city", values="count").fillna(0)
    ax = pivot.plot(kind="line", figsize=(10,4))
    ax.set_title("Price distribution (by bucket)")
    ax.set_xlabel("Price bucket")
    ax.set_ylabel("Count")
    plt.show()
else:
    print("ℹ️ Нема даних для гістограми цін.")

# 2) Occupancy by month (лінії по містах)
if occ_month is not None and occ_month.count() > 0:
    pdf = occ_month.toPandas().sort_values(["city","month"])
    pivot = pdf.pivot(index="month", columns="city", values="occ_rate_month")
    ax = pivot.plot(kind="line", figsize=(10,4))
    ax.set_title("Monthly occupancy")
    ax.set_xlabel("Month")
    ax.set_ylabel("Occupancy rate")
    plt.show()
else:
    print("ℹ️ Нема даних для помісячної зайнятості.")

# 3) Room type distribution (stacked bar, частки)
if room_share is not None and room_share.count() > 0:
    pdf = room_share.toPandas()
    pivot = pdf.pivot(index="room_type", columns="city", values="share").fillna(0)
    ax = pivot.plot(kind="bar", stacked=False, figsize=(10,4))
    ax.set_title("Room type share")
    ax.set_xlabel("Room type")
    ax.set_ylabel("Share")
    plt.xticks(rotation=0)
    plt.show()
else:
    print("ℹ️ Нема даних для розподілу типів кімнат.")

# 4) Ratings histogram (бар по біннах)
if rating_hist is not None and rating_hist.count() > 0:
    pdf = rating_hist.toPandas().sort_values(["city","rating_bin"])
    pivot = pdf.pivot(index="rating_bin", columns="city", values="count").fillna(0)
    ax = pivot.plot(kind="bar", figsize=(10,4))
    ax.set_title("Ratings distribution (bins of 5)")
    ax.set_xlabel("Rating bin")
    ax.set_ylabel("Count")
    plt.xticks(rotation=0)
    plt.show()
else:
    print("ℹ️ Нема даних для розподілу рейтингів.")
